### Derive the second derivative of the Gaussian function

Assume $\mu = 0$ for simplicity (or at least we have done the previous centered around 0)

The Gaussian function is given by:

$$g(x) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}$$


To differentiate $g(x)$ \), I first pull out the constant:

$$g'(x) = \frac{1}{\sqrt{2\pi\sigma^2}} \frac{d}{dx} \bigg[ e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)} \bigg]$$


Using the chain rule:


$$g'(x) = \frac{1}{\sqrt{2\pi\sigma^2}}\frac{d}{dx} e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}
= e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)} \cdot \frac{d}{dx} \left( -\frac{1}{2} \frac{x^2}{\sigma^2} \right)$$

Differentiating the exponent:

$$\frac{d}{dx} \left( -\frac{1}{2} \frac{x^2}{\sigma^2} \right) = -\frac{2x}{2\sigma^2} = -\frac{x}{\sigma^2}$$

Putting it all together gives:

$$g'(x) = \frac{1}{\sqrt{2\pi\sigma^2}} \cdot \left( -\frac{x}{\sigma^2} \right) e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}$$



Now so I don't need to write the constant around I let $c = \frac{1}{\sqrt{2\pi\sigma^2}}$ and write:


$$g'(x) = c \left( -\frac{x}{\sigma^2} \right) e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}$$

Now, differentiating again and using the product rule:


$$g''(x) = c \left[ \frac{d}{dx} \left( -\frac{x}{\sigma^2} \right) e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}
+ \left( -\frac{x}{\sigma^2} \right) \frac{d}{dx} e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}\right]$$

The first term:


$$\frac{d}{dx} \left( -\frac{x}{\sigma^2} \right) = -\frac{1}{\sigma^2}$$

Which is now:

$$ \left( -\frac{1}{\sigma^2} \right) e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}$$

For the second term, using the chain rule again:


$$\frac{d}{dx} e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}
= e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)} \cdot \frac{d}{dx} \left( -\frac{1}{2} \frac{x^2}{\sigma^2} \right)
$$

$$
= e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)} \left( -\frac{x}{\sigma^2} \right)
$$

And I obtain:

$$
\left( -\frac{x}{\sigma^2} \right) \left( -\frac{x}{\sigma^2} \right) e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}
= \left( \frac{x^2}{\sigma^4} \right) e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}
$$

Finally putting it all together and factoring out the exponential term:

$$
g''(x) = c \left[ \left( -\frac{1}{\sigma^2} \right) e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}
+ c \left( \frac{x^2}{\sigma^4} \right) e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}\right]
$$

$$
= c \left( \frac{x^2}{\sigma^4} - \frac{1}{\sigma^2} \right) e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}
$$

Rewriting the constant:

$$
g''(x) = \frac{1}{\sqrt{2\pi\sigma^2}} \left( \frac{x^2}{\sigma^4} - \frac{1}{\sigma^2} \right) e^{-\frac{1}{2} \left( \frac{x^2}{\sigma^2} \right)}
$$